In [4]:
!pip install -q google-generativeai

import os
import pandas as pd
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="models/gemini-2.5-flash",
  generation_config=generation_config
)

chat_session = model.start_chat(
    history=[
        {
            "role": "user",
            "parts": [
                "You are a market research analyst specializing in wireless earbuds.\n",
            ],
        }
    ]
)

response1 = chat_session.send_message("What are the key factors customers consider when buying wireless earbuds?")
print(response1.text)

response2 = chat_session.send_message("How would you analyze customer feedback about sound quality, fit/comfort, battery life, and connectivity in wireless earbuds?")
print(response2.text)

df = pd.read_csv("sentiment_analysis_results.csv")
df.head()

reviews = df["content"].fillna("").astype(str).head(20).tolist()

def analyze_reviews(reviews, api_key, prompt_message):
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel(
        model_name="models/gemini-2.5-flash",
        generation_config={
            "temperature": 1,
            "top_p": 0.95,
            "top_k": 64,
            "max_output_tokens": 8192,
            "response_mime_type": "text/plain",
        },
    )

    chat_session = model.start_chat(history=[])

    prompt = prompt_message + "\n\n".join(reviews)
    response = chat_session.send_message(prompt)
    return response.text

prompt_message = "Analyze the following wireless earbud reviews and summarize the key insights:"
insights = analyze_reviews(reviews, os.getenv("GOOGLE_API_KEY"), prompt_message)
print(insights)

def analyze_basic_product(reviews, product_name):
    prompt_message = f"""Analyze these reviews for {product_name}:
    1. What are the top 3 most mentioned features?
    2. What do customers love most about this product?
    3. What are the most common complaints?

    Please provide specific examples from the reviews."""
    return analyze_reviews(reviews, os.getenv("GOOGLE_API_KEY"), prompt_message)

product_reviews = df[df["product_name"].astype(str).str.contains("Beats Studio Buds", case=False, na=False)]["content"].fillna("").astype(str).tolist()[:30]
basic_insights = analyze_basic_product(product_reviews, "Beats Studio Buds")
print(basic_insights)

def analyze_sentiment_patterns(reviews, sentiment_category, product_name):
    prompt_message = f"""Analyze these {sentiment_category.lower()} reviews for {product_name}:
    1. What specific aspects create {sentiment_category.lower()} reactions?
    2. Identify common phrases or descriptions used
    3. Note any comparisons with competitor products
    4. Find unexpected or unique feedback

    Support your analysis with specific examples."""
    return analyze_reviews(reviews, os.getenv("GOOGLE_API_KEY"), prompt_message)

positive_reviews = df[
    (df["product_name"].astype(str).str.contains("Beats Studio Buds", case=False, na=False)) &
    (df["Sentiment"] == "Positive")
]["content"].fillna("").astype(str).tolist()[:30]

negative_reviews = df[
    (df["product_name"].astype(str).str.contains("Beats Studio Buds", case=False, na=False)) &
    (df["Sentiment"] == "Negative")
]["content"].fillna("").astype(str).tolist()[:30]

positive_insights = analyze_sentiment_patterns(positive_reviews, "Positive", "Beats Studio Buds")
print(positive_insights)

negative_insights = analyze_sentiment_patterns(negative_reviews, "Negative", "Beats Studio Buds")
print(negative_insights)

beats_reviews = df[df["product_name"].astype(str).str.contains("Beats Studio Buds", case=False, na=False)]["content"].fillna("").astype(str).tolist()[:20]
airpods_reviews = df[df["product_name"].astype(str).str.contains("Apple AirPods", case=False, na=False)]["content"].fillna("").astype(str).tolist()[:20]

products_to_compare = {
    "Beats Studio Buds": beats_reviews,
    "Apple AirPods": airpods_reviews
}

def analyze_comparative_reviews(products_to_compare):
    insights = {}
    for product, reviews in products_to_compare.items():
        prompt_message = f"Summarize customer opinions about {product} (sound, fit, battery, connectivity) with pros/cons:"
        insights[product] = analyze_reviews(reviews, os.getenv("GOOGLE_API_KEY"), prompt_message)
    return insights

comparison_insights = analyze_comparative_reviews(products_to_compare)
print(comparison_insights)

def analyze_feature(reviews, feature_name):
    prompt_message = f"""Analyze mentions of {feature_name} in these reviews:
    1. How do customers describe the {feature_name}?
    2. What specific aspects are praised or criticized?
    3. How does {feature_name} compare across products (if mentioned)?
    4. What improvements do customers suggest?

    Include specific examples."""
    return analyze_reviews(reviews, os.getenv("GOOGLE_API_KEY"), prompt_message)

battery_reviews = df[df["content"].fillna("").astype(str).str.contains("battery|charge|charging|power", case=False, na=False)]["content"].fillna("").astype(str).tolist()[:40]
battery_insights = analyze_feature(battery_reviews, "battery life")
print(battery_insights)

As a market research analyst specializing in wireless earbuds, I can confidently say that customers consider a multi-faceted set of factors, often prioritizing differently based on their individual needs and budget. Here are the key factors, broken down:

### Key Factors Customers Consider When Buying Wireless Earbuds

1.  **Audio Performance (Sound Quality):**
    *   **Clarity & Fidelity:** Crisp highs, clear mids, and detailed sound reproduction.
    *   **Bass Response:** Punchy, deep, and well-controlled bass without being muddy.
    *   **Sound Profile:** Some prefer a neutral sound, others a more V-shaped (boosted bass and treble), or warm sound. Custom EQ options are a plus.
    *   **Spatial Audio/Immersive Sound:** For a more cinematic or concert-like experience (increasingly important for premium models).

2.  **Active Noise Cancellation (ANC) & Transparency Mode:**
    *   **ANC Effectiveness:** How well they block out ambient noise (e.g., airplane hum, city traffic, office